# 학습된 모델 가중치를 불러와 추론하기

- 학습이 끝난 모델의 저장된 가중치를 불러와 학습 데이터에 존재하지 않는 새로운 입력에 대해서도 예측값을 얻어 봅시다.
- 먼저 필요한 라이브러리들을 import 해줍니다.

In [1]:
!pip install transformers

import os
import requests
import torch
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

- koelectra-small 모델을 기반으로 학습한 모델의 가중치가 프로젝트 repo에 업로드되어 있습니다. 가중치 파일을 다운로드해 저장한 뒤 사용하겠습니다.
- 본인이 직접 학습한 모델의 가중치를 사용해도 무방합니다.

In [2]:
# download data
os.makedirs('./models', exist_ok=True)

model_url = "https://github.com/GirinMan/HAI-DialectTranslator/raw/main/classifier_training/model.pth"
model_response = requests.get(model_url, allow_redirects=True)

open('./models/model.pth', 'wb').write(model_response.content)   

56564173

- State dictionary는 모델의 종류나 구조는 저장하지 않고, 각 레이어의 가중치 값만 저장하기 때문에, 모델을 저장할 때 사용하였던 구조와 동일한 모델 객체를 불러와 사용해야 합니다.
- 만약 학습 과정에서 다른 체크포인트를 기반으로 학습했다면, 동일하게 변경해주어야 합니다.

In [5]:
model_ckpt = "monologg/koelectra-small-v3-discriminator"
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

- load_state_dict 함수를 이용해 가중치를 모델에 덮어씌워줍니다.

In [6]:
state = torch.load('./models/model.pth', map_location='cuda')
model.load_state_dict(state, strict=False)

<All keys matched successfully>

- 모델을 학습한 이후 테스트할 때 사용했던 코드와 유사한 infer 함수입니다.
- tokenizer를 통해 입력을 전처리하고, 모델에 입력한 결과값을 바탕으로 표준어/방언 여부를 출력해줍니다.

In [7]:
def infer(model, tokenizer, input_txt):
    input_tensor = torch.tensor([tokenizer.encode(input_txt)]).to('cuda')

    with torch.no_grad():
        preds = model(input_tensor).logits.cpu()

    result = np.argmax(preds, axis=1).item()

    print('입력된 문장 "' + input_txt + '"는 ', end='')
    if result:
        print("경상도 방언입니다.")
    else:
        print("표준어 발화입니다.")

- 이제 사용자로부터 입력을 받아 모델을 거쳐 입력된 발화가 표준어인지 경상도 방언인지 구분해내는 프로그램을 작성해 보겠습니다.

<img src="https://img2.quasarzone.com/editor/2021/08/20/88adb31eff6afe18af2121abc2252904.jpg" width="300">

In [12]:
# 종료하려면 빈 값을 입력하세요.
while True:
    input_txt = input("발화 텍스트를 입력하세요> ")
    if len(input_txt) == 0:
        break
    infer(model, tokenizer, input_txt)

발화 텍스트를 입력하세요> 마 니 소행성이가?
입력된 문장 "마 니 소행성이가?"는 경상도 방언입니다.
발화 텍스트를 입력하세요> 나 소행성 아이다!
입력된 문장 "나 소행성 아이다!"는 경상도 방언입니다.
발화 텍스트를 입력하세요> 야 너 소행성이야?
입력된 문장 "야 너 소행성이야?"는 표준어 발화입니다.
발화 텍스트를 입력하세요> 나 소행성 아니야!
입력된 문장 "나 소행성 아니야!"는 표준어 발화입니다.
발화 텍스트를 입력하세요> 
